# Making firmware with hls4ml

This is the fun part! Now we'll convert our Keras model into highly parallel FPGA firmware with hls4ml, using only a few lines of code.

<img src="images/hls4ml_conifer.png" alt="hls4ml" width="1000" img align="center"/>

Let's make bitfiles both of the large and the compressed model:

In [5]:
import hls4ml
import util
import h5py
import numpy as np
import tensorflow as tf

from qkeras.utils import _add_supported_quantized_objects
co = {}; _add_supported_quantized_objects(co)

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

with h5py.File('Ato4l_dataset.h5', 'r') as file:
    signal_test_data = np.array(file['Data'])

# First the baseline:
autoencoder = tf.keras.models.load_model('baseline_ae.h5')

config = hls4ml.utils.config_from_keras_model(autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'

for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64
hls_model = hls4ml.converters.convert_from_keras_model(autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator',
                                                         output_dir='baseline_ae_pynq',
                                                         board='pynq-z2')
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = []                                                     
hls_model.compile()

y_hls4ml = hls_model.predict(np.ascontiguousarray(signal_test_data))
hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)
util.package(hls_model, signal_test_data, y_hls4ml)

# Then the compressed model:
q_autoencoder = tf.keras.models.load_model('compressed_ae.h5')

config = hls4ml.utils.config_from_keras_model(q_autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64
q_hls_model = hls4ml.converters.convert_from_keras_model(q_autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator',
                                                         output_dir='
                                                         ',
                                                         board='pynq-z2')
q_hls_model.model.optimizer.OutputRoundingSaturationMode.layers = []                                                     
q_hls_model.compile()

y_q_hls4ml = q_hls_model.predict(np.ascontiguousarray(signal_test_data))
hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(q_hls_model)
util.package(q_hls_model, signal_test_data, y_q_hls4ml)


Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: dense, layer type: Dense
  -> Activation (linear), layer name: dense
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: leaky_re_lu, layer type: LeakyReLU
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: leaky_re_lu_1, layer type: LeakyReLU
Layer name: dense_2, layer type: Dense
  -> Activation (linear), layer name: dense_2
Layer name: dense_3, layer type: Dense
  -> Activation (linear), layer name: dense_3
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: leaky_re_lu_2, layer type: LeakyReLU
Layer name: dense_4, layer type: Dense
  -> Activation (linear), layer name: dense_4
Layer name: batch_normalization_4, layer type: BatchNormalization
Layer name: leaky_re_lu_3, layer type:

sh: vivado_hls: command not found
sh: vivado: command not found


Project myproject_prj does not exist. Rerun "hls4ml build -p baseline_ae_pynq".
File baseline_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper.bit not found, waiting 60s before retry


KeyboardInterrupt: 

In [ ]:
import os

files = os.listdir('.')
print(files)